In [2]:
import pandas as pd
import numpy as np
from collections import *
from itertools import islice
from datetime import datetime
import pickle
import os
import time
import csv
import logging
import threading
from multiprocessing import Process, Pool, Manager
import multiprocessing as mp
import os

In [3]:
dates = os.listdir('./logs/')

In [4]:
len(dates)

61

In [5]:
with open('./logs/2019-08-02/conn.00:00:00-01:00:00.log') as f:
    f = f.readlines()
for line in f:
    if line.split("\t")[0]=='#fields':
        columns = line.split("\t")[1:]
        for i, c in enumerate(columns):
            if len(c.split("\n"))>1:
                columns[i] = c.split("\n")[0]
        break

In [6]:
with open('logs/2019-06-22/weird.00:00:00-01:00:00.log') as f:
    f = f.readlines()
for line in f:
    if line.split("\t")[0]=='#fields':
        w_columns = line.split("\t")[1:]
        for i, c in enumerate(w_columns):
            if len(c.split("\n"))>1:
                w_columns[i] = c.split("\n")[0]
        break

In [7]:
weird_lst = ['weird.00:00:00-01:00:00.log', 'weird.01:00:00-02:00:00.log', 'weird.02:00:00-03:00:00.log', 'weird.03:00:00-04:00:00.log',\
            'weird.04:00:00-05:00:00.log', 'weird.05:00:00-06:00:00.log', 'weird.06:00:00-07:00:00.log', 'weird.07:00:00-08:00:00.log',\
            'weird.08:00:00-09:00:00.log', 'weird.09:00:00-10:00:00.log', 'weird.10:00:00-11:00:00.log', 'weird.11:00:00-12:00:00.log',\
            'weird.12:00:00-13:00:00.log', 'weird.13:00:00-14:00:00.log', 'weird.14:00:00-15:00:00.log', 'weird.15:00:00-16:00:00.log',\
            'weird.16:00:00-17:00:00.log', 'weird.17:00:00-18:00:00.log', 'weird.18:00:00-19:00:00.log', 'weird.19:00:00-20:00:00.log',\
            'weird.20:00:00-21:00:00.log', 'weird.21:00:00-22:00:00.log', 'weird.22:00:00-23:00:00.log', 'weird.23:00:00-00:00:00.log']

In [8]:
# ilts = 0
# iil = 0
# nil = 0
# for d in dates:
#     for i in weird_lst:
#         try:
#             weird_df = pd.read_csv('logs/'+ d + '/' + i, delimiter="\t", header=None, names=w_columns, skiprows=lambda x:x in range(8), skipfooter=1)
#             weird_cnt = Counter(weird_df.name)
#             ilts += weird_cnt['irc_line_too_short']
#             iil += weird_cnt['irc_invalid_line']
#             nil += weird_cnt['NUL_in_line']
#         except:
#             print(d, i)
#     print(d, ilts, iil)

In [9]:
ilts = 0
iil = 0
nil = 0
dfs= []
for i in weird_lst:
    weird_df = pd.read_csv('logs/'+'2019-08-02/' + i, delimiter="\t", header=None, names=w_columns, skiprows=lambda x:x in range(8), skipfooter=1)
    weird_cnt = Counter(weird_df.name)
    ilts += weird_cnt['irc_line_too_short']
    iil += weird_cnt['irc_invalid_line']
    nil += weird_cnt['NUL_in_line']
    dfs.append(weird_df)

/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [10]:
total_weird = pd.concat(dfs)

In [11]:
len(total_weird)

1715431

In [12]:
ilts_weird = total_weird[total_weird.name=='irc_line_too_short']

In [13]:
iil_weird = total_weird[total_weird.name=='irc_invalid_line']

In [14]:
ilts_weird["date"] = pd.to_datetime(ilts_weird["ts"], unit='s')

/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
iwip = ilts_weird['id.orig_h'].unique().tolist()
iiwip = iil_weird["id.orig_h"].unique().tolist()

In [16]:
iwips = list(set(iwip).union(set(iiwip)))

In [17]:
iwips

['203.235.97.82',
 '103.24.8.17',
 '211.239.236.24',
 '1.11.71.189',
 '203.235.97.150',
 '203.235.44.150',
 '103.24.8.19',
 '211.239.236.17',
 '211.239.236.18',
 '103.24.8.22',
 '210.112.7.7',
 '203.235.98.160',
 '210.218.197.135',
 '14.0.73.80',
 '210.218.197.106',
 '211.239.236.9',
 '203.235.97.103',
 '210.112.7.5']

In [18]:
uids = ilts_weird["uid"]

In [19]:
total_weird[total_weird["id.orig_h"].isin(iwip)]

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,name,addl,notice,peer
284,1.564672e+09,CHJ75E37r9qFQcR1D,210.218.197.135,63643,61.248.190.144,80,possible_split_routing,-,F,zeek
285,1.564672e+09,CHJ75E37r9qFQcR1D,210.218.197.135,63643,61.248.190.144,80,data_before_established,-,F,zeek
1222,1.564672e+09,Cg1dS64ezhLv8Gat19,210.218.197.135,63657,61.248.190.144,80,possible_split_routing,-,F,zeek
1223,1.564672e+09,Cg1dS64ezhLv8Gat19,210.218.197.135,63657,61.248.190.144,80,data_before_established,-,F,zeek
1959,1.564672e+09,C8d9vb2zn7e9t2PQn5,210.218.197.135,63667,61.248.190.144,80,possible_split_routing,-,F,zeek
1960,1.564672e+09,C8d9vb2zn7e9t2PQn5,210.218.197.135,63667,61.248.190.144,80,data_before_established,-,F,zeek
2714,1.564672e+09,ClxUDn3EuTt0CCAiP2,210.218.197.135,63688,61.248.190.144,80,possible_split_routing,-,F,zeek
2715,1.564672e+09,ClxUDn3EuTt0CCAiP2,210.218.197.135,63688,61.248.190.144,80,data_before_established,-,F,zeek
3321,1.564672e+09,CWfP4d1ayKCcAVcAcc,210.218.197.135,63708,61.248.190.144,80,possible_split_routing,-,F,zeek
3322,1.564672e+09,CWfP4d1ayKCcAVcAcc,210.218.197.135,63708,61.248.190.144,80,data_before_established,-,F,zeek


In [20]:
conn_lst = ['conn.00:00:00-01:00:00.log', 'conn.01:00:00-02:00:00.log', 'conn.02:00:00-03:00:00.log', 'conn.03:00:00-04:00:00.log',\
            'conn.04:00:00-05:00:00.log', 'conn.05:00:00-06:00:00.log', 'conn.06:00:00-07:00:00.log', 'conn.07:00:00-08:00:00.log',\
            'conn.08:00:00-09:00:00.log', 'conn.09:00:00-10:00:00.log', 'conn.10:00:00-11:00:00.log', 'conn.11:00:00-12:00:00.log',\
            'conn.12:00:00-13:00:00.log', 'conn.13:00:00-14:00:00.log', 'conn.14:00:00-15:00:00.log', 'conn.15:00:00-16:00:00.log',\
            'conn.16:00:00-17:00:00.log', 'conn.17:00:00-18:00:00.log', 'conn.18:00:00-19:00:00.log', 'conn.19:00:00-20:00:00.log',\
            'conn.20:00:00-21:00:00.log', 'conn.21:00:00-22:00:00.log', 'conn.22:00:00-23:00:00.log', 'conn.23:00:00-00:00:00.log']

In [21]:
ilts = 0
iil = 0
nil = 0
dfs= []
for i in conn_lst:
    df = pd.read_csv('logs/'+'2019-08-02/' + i, delimiter="\t", header=None, names=columns, skiprows=lambda x:x in range(8), skipfooter=1)
    dfs.append(df)

/clusterfs/csdata/pkg/anaconda3/envs/exagpu/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [22]:
conn_df = pd.concat(dfs)

In [23]:
conn_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents
0,1.564672e+09,Ck2tmu4lxEMJFclUP5,121.55.166.4,28612,203.250.220.176,23,tcp,-,-,-,...,S0,F,F,0,S,1,40,0,0,-
1,1.564672e+09,CVwuBp4ZVHMjXrFAVh,124.195.191.125,35299,150.197.90.231,23,tcp,-,-,-,...,S0,F,F,0,S,1,40,0,0,-
2,1.564672e+09,CDtZ001AQWfFa2asgd,134.75.122.2,18372,211.239.236.29,53,udp,dns,-,-,...,SHR,F,F,0,^d,0,0,1,97,-
3,1.564671e+09,C6c3Ri1JlDHHQa0d69,211.225.30.126,554,210.116.238.135,51916,tcp,-,5.357130,11710,...,SH,F,F,0,DCAF,8,6190,0,0,-
4,1.564672e+09,Cxojly1K5hrx08npVb,203.241.124.174,47540,211.115.202.155,50505,udp,-,-,-,...,S0,F,F,0,D,1,329,0,0,-


In [24]:
conn_df["class"]=0

In [25]:
conn_df.loc[conn_df["id.orig_h"].isin(iwip),["class"]] = 'abnorm'
# total_df.loc[total_df["id.orig_h"].isin(infected),:]["class"] = "botnet"
# total_df.loc[total_df["id.orig_h"].isin(infected),["class"]] = "botnet"

In [26]:
conn_df.loc[~conn_df["id.orig_h"].isin(iwip),["class"]] = 'norm'

In [27]:
conn_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,class
0,1.564672e+09,Ck2tmu4lxEMJFclUP5,121.55.166.4,28612,203.250.220.176,23,tcp,-,-,-,...,F,F,0,S,1,40,0,0,-,norm
1,1.564672e+09,CVwuBp4ZVHMjXrFAVh,124.195.191.125,35299,150.197.90.231,23,tcp,-,-,-,...,F,F,0,S,1,40,0,0,-,norm
2,1.564672e+09,CDtZ001AQWfFa2asgd,134.75.122.2,18372,211.239.236.29,53,udp,dns,-,-,...,F,F,0,^d,0,0,1,97,-,norm
3,1.564671e+09,C6c3Ri1JlDHHQa0d69,211.225.30.126,554,210.116.238.135,51916,tcp,-,5.357130,11710,...,F,F,0,DCAF,8,6190,0,0,-,norm
4,1.564672e+09,Cxojly1K5hrx08npVb,203.241.124.174,47540,211.115.202.155,50505,udp,-,-,-,...,F,F,0,D,1,329,0,0,-,norm


In [28]:
conn_df.reset_index(inplace=True)

In [29]:
Counter(conn_df["class"])

Counter({'norm': 7128603, 'abnorm': 5446})

In [30]:
# conn_df.to_csv('kisti_bro_result.csv')

In [31]:
def preprocess_stat_3(df, period_len, rm_ntp):
#     df = df.loc[~df["service"].isin(['mysql', 'imap','ftp','ftp-data'])]
    df["orig_bytes"] = df["orig_bytes"].apply(lambda x:0 if x=='-' else x)
    df["resp_bytes"] = df["orig_bytes"].apply(lambda x:0 if x=='-' else x)
    df["orig_bytes"] = df["orig_bytes"].astype('int64')
    df["resp_bytes"] = df["resp_bytes"].astype('int64')
    if rm_ntp:
        df = df.loc[~df["service"].isin(['ntp'])]
    df = df.sort_values(by='ts')
    df = df.reset_index(drop=True)
    df["StartTime"] = pd.to_datetime(df.ts, unit='s')
    df["min"] = df["StartTime"].apply(lambda x: x.minute)
    df["sec"] = df["StartTime"].apply(lambda x: x.second)
    df["hour"] = df["StartTime"].apply(lambda x: x.hour)
    df["time_bin"] = df.apply(lambda x:int(x["hour"]) * 60 * 60 +int(x["min"]) * 60 + int(x["sec"]), axis=1)
    tb_lst = df.time_bin.unique()
    tb_dict = dict()
    for i in tb_lst:
        tb_dict[i] = (i - df.time_bin.min())//period_len
    df["time_chunk"] = df["time_bin"].apply(lambda x:tb_dict[x])
    # df = port_service_rate(df)
    df_class = df.filter(["time_chunk", "id.orig_h", "class"], axis=1)
    df.duration = df.apply(lambda x: 0 if x.duration =='-' else x.duration, axis=1)
    df.duration = df.duration.astype(float)
    df_filtered = df.filter(['duration','orig_pkts','orig_ip_bytes','orig_bytes', 'resp_bytes',\
                             'missed_bytes','resp_pkts', 'resp_ip_bytes','proto','service',\
                             "time_chunk",'id.orig_h', 'id.resp_p', 'id.orig_p','id.resp_h'], axis=1)
    
    df_dport = df_filtered.groupby(["time_chunk","id.orig_h"])["id.resp_p"].nunique().reset_index(level=[0,1])
    df_dstaddr = df_filtered.groupby(["time_chunk","id.orig_h"])["id.resp_h"].nunique().reset_index(level=[0,1])
    df_sport = df_filtered.groupby(["time_chunk","id.orig_h"])["id.orig_p"].nunique().reset_index(level=[0,1])
    
    grouped = df_filtered.groupby(["time_chunk","id.orig_h"])
    df_ipcnt = pd.DataFrame(grouped["id.orig_h"].agg('count')).rename(columns={'id.orig_h':"sripcnt"}).reset_index()
    result = grouped['duration','orig_pkts','orig_ip_bytes',\
                     'missed_bytes','resp_pkts','resp_ip_bytes','orig_bytes','resp_bytes'].aggregate([np.mean, np.std])
    result_service = df_filtered.groupby(["time_chunk","id.orig_h","service"]).size().unstack(fill_value=0).reset_index(level=[0,1])
    result_service = result_service.drop(["-"], axis=1)
    result_proto = df_filtered.groupby(["time_chunk","id.orig_h","proto"]).size().unstack(fill_value=0).reset_index(level=[0,1])
    # result_state = df_filtered.groupby(["time_chunk","SrcAddr","State"]).size().unstack(fill_value=0).reset_index(level=[0,1])
    
    df_merged = result.reset_index(level=[0,1])
    df_merged.columns = [' '.join(col).strip() for col in df_merged.columns.values]
    df_merged = df_merged.drop_duplicates()
    df_merged = df_merged.fillna(0)
    df_merged = df_merged.merge(df_class.drop_duplicates(subset=["id.orig_h","time_chunk"]), how='left')
    df_final = result_service.merge(df_merged)
    df_final = df_final.merge(result_proto)
    df_final = df_final.merge(df_dport)
    df_final = df_final.merge(df_dstaddr)
    df_final = df_final.merge(df_sport)
    df_final = df_final.merge(df_ipcnt)
    # df_final = df_final.merge(result_state)
    df_fill_oh = df_final.dropna()
    normalize_column = ["duration mean","duration std", "orig_pkts mean", \
                        "orig_pkts std", "orig_ip_bytes mean", "orig_ip_bytes std",\
                        "orig_bytes mean", "orig_bytes std", "resp_bytes mean", "resp_bytes std",\
                        "missed_bytes mean", "missed_bytes std",\
                        "resp_pkts mean", "resp_pkts std", "resp_ip_bytes mean", "resp_ip_bytes std"]\
                        + ["id.resp_p", "id.resp_h","id.orig_p","sripcnt"]
    normalize_column = [x for x in normalize_column if str(x) != 'nan']
    for i in normalize_column:
        df_fill_oh[i] = (df_fill_oh[i] - df_fill_oh[i].min())/(df_fill_oh[i].max()-df_fill_oh[i].min())

    normalize_column = list(df.service.unique())
    normalize_column.remove("-")
    normalize_column = [x for x in normalize_column if str(x) != 'nan']
    df_fill_oh.loc[:, normalize_column] =\
             df_fill_oh.loc[:,normalize_column].div(df_fill_oh.loc[:,normalize_column].sum(axis=1), axis=0)
    normalize_column = ['icmp', 'tcp', 'udp', 'arp', 'igmp', 'rtp']
    for c in normalize_column:
        if c not in df_fill_oh.columns:
            df_fill_oh[c] = 0
    df_fill_oh.loc[:, normalize_column] =\
             df_fill_oh.loc[:,normalize_column].div(df_fill_oh.loc[:,normalize_column].sum(axis=1), axis=0)
    df_fill_oh = df_fill_oh.fillna(0)
    df_fill_oh = df_fill_oh.sort_index(axis=1)
    return df_fill_oh

In [33]:
a = preprocess_stat_3(conn_df, 5, False)

In [ ]:
a=Counter(conn_df["class"])

In [72]:
len(conn_df[conn_df["id.orig_h"].isin(iwip)])

5446

In [53]:
Counter(conn_df[conn_df["id.orig_h"].isin(iwip)]["service"])

Counter({'-': 3984, 'http': 1426, 'irc': 34, 'ftp': 2})

In [54]:
len(conn_df)

7134049

In [39]:
iwip

array(['210.218.197.106', '210.218.197.135', '14.0.73.80',
       '211.239.236.18', '211.239.236.17', '210.112.7.7', '203.235.97.82',
       '103.24.8.17', '103.24.8.19', '1.11.71.189', '203.235.97.150',
       '203.235.97.103', '211.239.236.9', '103.24.8.22', '203.235.44.150',
       '210.112.7.5', '203.235.98.160', '211.239.236.24'], dtype=object)